# TCGA-UCEC – Notebook 0A
## Acquisition des données brutes (GDC/TCGA)

**Objectif** : Télécharger les données de comptage STAR et les métadonnées cliniques via l'API du GDC en utilisant l'interopérabilité R-Python. Cette étape garantit la provenance et la reproductibilité des données sources.

## 1. Configuration et Environnement

In [ ]:
# ==========================================================================================================
import os                                       # Navigation fichiers (DIRS, chemins relatifs)
import sys                                      # Interaction système
import pandas as pd                             # Manipulation de tables
import subprocess                               # Exécution de commandes avec streaming de sortie

from datetime import datetime

# ==========================================================================================================
# Définition de la racine du projet
PROJECT_ROOT = os.getcwd()

DIRS = {
    "RAW":       os.path.join(PROJECT_ROOT, "data", "raw"),
    "SCRIPTS":   os.path.join(PROJECT_ROOT, "scripts"),
    "PROCESSED": os.path.join(PROJECT_ROOT, "data", "processed")
}

for path in DIRS.values():
    os.makedirs(path, exist_ok=True)

# ==========================================================================================================
print(f"✅ Environnement configuré. Racine : {PROJECT_ROOT}")
print(f"✅ Dossiers d'acquisition prêts.")

✅ Environnement configuré. Racine : c:\Z\M2_AIDA\TCGA_UCEC_project
✅ Dossiers d'acquisition prêts.


## 2. Exécution du script d'acquisition (Rscript)

Le téléchargement s'appuie sur la librairie R **TCGAbiolinks**. Nous appelons le script fourni directement depuis ce notebook pour maintenir une traçabilité complète.

In [2]:
# ==========================================================================================================
# Chemin vers l'exécutable R et le script moteur
R_EXECUTABLE = r"C:\Program Files\R\R-4.5.2\bin\x64\Rscript.exe"
r_script_path = os.path.join(DIRS["SCRIPTS"], "Script_data_download.R")

if os.path.exists(r_script_path):
    print("⏳ Acquisition en cours : téléchargement depuis le GDC (cela peut prendre 5-10 minutes)...\n")
    
    # Appel système à Rscript
    # Utilisation de subprocess pour capturer et afficher la sortie de R en temps réel
    process = subprocess.Popen(
        [R_EXECUTABLE, r_script_path], 
        stdout=subprocess.PIPE, 
        stderr=subprocess.STDOUT, 
        text=True,
        encoding='utf-8'
    )

    # Lecture de la sortie ligne par ligne pour un retour utilisateur immédiat
    for line in process.stdout:
        print(line, end='')

    process.wait()
    
    if process.returncode == 0:
        print("\n✅ Script R terminé avec succès. Données prêtes dans data/raw/.")
    else:
        print(f"\nErreur lors de l'exécution du script R (Code retour: {process.returncode})")
else:
    print(f"Fichier script R introuvable : {r_script_path}")

⏳ Acquisition en cours : téléchargement depuis le GDC (cela peut prendre 5-10 minutes)...

Le chargement a nécessité le package : MatrixGenerics
Le chargement a nécessité le package : matrixStats

Attachement du package : 'MatrixGenerics'

Les objets suivants sont masqués depuis 'package:matrixStats':

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2,

## 2Bis. Relocalisation des fichiers (Workaround Post-Acquisition)

Le script R exporte par défaut dans un dossier nommé tcga_ucec_counts_export. Nous déplaçons ces fichiers vers data/raw/ pour respecter l'arborescence du projet et permettre aux notebooks suivants d'accéder aux données.

In [ ]:
import shutil

# Chemins source et cible (os est déjà importé en cellule 1)
source_dir = os.path.join(PROJECT_ROOT, "tcga_ucec_counts_export")
target_dir = DIRS["RAW"]

files_to_move = [
    "counts_samples_x_genes.csv.gz",
    "metadata_clinical_merged.csv.gz",
    "gene_annotation.csv.gz"
]

if os.path.exists(source_dir):
    print(f"📂 Dossier source détecté : {source_dir}")
    moved_count = 0

    for f in files_to_move:
        src_path = os.path.join(source_dir, f)
        dst_path = os.path.join(target_dir, f)
        
        if os.path.exists(src_path):
            shutil.move(src_path, dst_path)
            print(f"✅ Déplacé : {f}")
            moved_count += 1
            
    if moved_count > 0:
        print(f"\n✅ Harmonisation terminée. {moved_count} fichiers déplacés vers {target_dir}.")
else:
    # Vérification de sécurité si les fichiers sont déjà là (évite de planter au re-run)
    if all(os.path.exists(os.path.join(target_dir, f)) for f in files_to_move):
        print("✅ Les fichiers sont déjà correctement placés dans data/raw/.")
    else:
        print(f"❌ Erreur : Dossier source {source_dir} introuvable et fichiers absents de {target_dir}.")

📂 Dossier source détecté : c:\Z\M2_AIDA\TCGA_UCEC_project\tcga_ucec_counts_export
✅ Déplacé : counts_samples_x_genes.csv.gz
✅ Déplacé : metadata_clinical_merged.csv.gz
✅ Déplacé : gene_annotation.csv.gz

✅ Harmonisation terminée. 3 fichiers déplacés vers c:\Z\M2_AIDA\TCGA_UCEC_project\data\raw.


## 3. Vérification des fichiers téléchargés

Contrôle de la présence physique des fichiers dans `data/raw/` et affichage des dimensions brutes.

In [9]:
# ==========================================================================================================
# Définition des chemins vers les fichiers attendus dans data/raw/
expr_file = os.path.join(DIRS["RAW"], "counts_samples_x_genes.csv.gz")
meta_file = os.path.join(DIRS["RAW"], "metadata_clinical_merged.csv.gz")
gene_file = os.path.join(DIRS["RAW"], "gene_annotation.csv.gz")

# Vérification de l'existence et chargement des données brutes
if os.path.exists(expr_file) and os.path.exists(meta_file) and os.path.exists(gene_file):
    print("Lecture des fichiers compressés (cela peut prendre quelques secondes)...")
    
    # Lecture avec pandas (gestion native du .gz)
    # index_col=0 car la première colonne contient les barcodes/identifiants
    df_expr = pd.read_csv(expr_file, index_col=0)
    df_meta = pd.read_csv(meta_file, index_col=0)
    df_gene = pd.read_csv(gene_file, index_col=0)
    
    print(f"\n✅ Vérification terminée avec succès.")
    print(f"   - Matrice de Counts  ({os.path.basename(expr_file)})  : {df_expr.shape[0]} échantillons x {df_expr.shape[1]} gènes")
    print(f"   - Métadonnées        ({os.path.basename(meta_file)})  : {df_meta.shape[0]} échantillons x {df_meta.shape[1]} colonnes cliniques")
    print(f"   - Informations Gènes ({os.path.basename(gene_file)})  : {df_gene.shape[0]} gènes x {df_gene.shape[1]} colonnes annotées\n")
    
    # Affichage des premières lignes pour inspection visuelle
    display(df_expr.head(3))
    display(df_meta.head(3))
    display(df_gene.head(3))

else:
    print(f"❌ Erreur : Fichiers introuvables dans {DIRS['RAW']}.")
    print("Veuillez vérifier que la cellule 2Bis (Relocalisation) a bien été exécutée.")

Lecture des fichiers compressés (cela peut prendre quelques secondes)...

✅ Vérification terminée avec succès.
   - Matrice de Counts  (counts_samples_x_genes.csv.gz)  : 553 échantillons x 60660 gènes
   - Métadonnées        (metadata_clinical_merged.csv.gz)  : 553 échantillons x 175 colonnes cliniques
   - Informations Gènes (gene_annotation.csv.gz)  : 60660 gènes x 2 colonnes annotées



,ENSG00000000003.15,ENSG00000000005.6,ENSG00000000419.13,ENSG00000000457.14,ENSG00000000460.17,ENSG00000000938.13,ENSG00000000971.16,ENSG00000001036.14,ENSG00000001084.13,ENSG00000001167.14,...,ENSG00000288661.1,ENSG00000288662.1,ENSG00000288663.1,ENSG00000288665.1,ENSG00000288667.1,ENSG00000288669.1,ENSG00000288670.1,ENSG00000288671.1,ENSG00000288674.1,ENSG00000288675.1
sample_barcode,,,,,,,,,,,,,,,,,,,,,
TCGA-EY-A1GK-01A-11R-A13S-07,1904,2,607,529,138,167,238,1427,597,877,...,0,1,20,0,0,0,68,0,2,44
TCGA-FI-A2CX-01A-11R-A17B-07,496,1,839,253,194,4301,847,3321,2903,1147,...,0,1,18,0,0,3,85,0,9,7
TCGA-AX-A3G3-01A-11R-A213-07,2115,2,2657,352,284,673,952,1760,1092,1317,...,0,0,7,0,0,2,101,0,13,129


,patient,sample,shortLetterCode,definition,sample_submitter_id,tumor_descriptor,sample_id,pathology_report_uuid,submitter_id,sample_type,...,treatments_radiation_treatment_dose,treatments_radiation_number_of_fractions,treatments_radiation_treatment_anatomic_sites,treatments_radiation_treatment_dose_units,treatments_radiation_prescribed_dose_units,treatments_radiation_route_of_administration,treatments_radiation_number_of_cycles,treatments_radiation_prescribed_dose,treatments_radiation_residual_disease,bcr_patient_barcode.y
barcode,,,,,,,,,,,,,,,,,,,,,
TCGA-EY-A1GK-01A-11R-A13S-07,TCGA-EY-A1GK,TCGA-EY-A1GK-01A,TP,Primary solid Tumor,TCGA-EY-A1GK-01A,Primary,d4e702a9-8e4a-4ad7-8b45-691c95dcf298,B5024D10-E23D-4B3B-8C54-13EBAA423A04,TCGA-EY-A1GK,Primary Tumor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TCGA-EY-A1GK
TCGA-FI-A2CX-01A-11R-A17B-07,TCGA-FI-A2CX,TCGA-FI-A2CX-01A,TP,Primary solid Tumor,TCGA-FI-A2CX-01A,Primary,b6d24830-0d69-44d7-94cf-7dfdc49e52c4,0F6F56E4-C4F9-4915-BE4E-1FBF4C6D93AD,TCGA-FI-A2CX,Primary Tumor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TCGA-FI-A2CX
TCGA-AX-A3G3-01A-11R-A213-07,TCGA-AX-A3G3,TCGA-AX-A3G3-01A,TP,Primary solid Tumor,TCGA-AX-A3G3-01A,Primary,5efdace8-5790-4239-8e8a-6349f88b10dd,3D86258F-2C9B-4DF2-86C5-E12B3FCB480D,TCGA-AX-A3G3,Primary Tumor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TCGA-AX-A3G3


,gene_symbol,gene_type
ensembl_gene_id,,
ENSG00000000003.15,TSPAN6,protein_coding
ENSG00000000005.6,TNMD,protein_coding
ENSG00000000419.13,DPM1,protein_coding


### Guide de Référence : Les Barcodes TCGA
<Small>
L'identifiant TCGA (Barcode) est la clé de voûte de l'organisation des données. Il permet de remonter de la donnée brute (séquençage) jusqu'à l'entité biologique (le patient).

1. Le Barcode Long (Échantillon / Sample)
Ce code complet est utilisé dans la Matrice d'Expression. Il détaille précisément l'origine technique de la donnée.
<Br>Projet (TCGA) : The Cancer Genome Atlas.
<Br>TSS (EY) : Tissue Source Site (le centre où le prélèvement a été effectué).
<Br>Participant (A1GK) : ID Patient (identifiant unique de la patiente, clé à 12 caractères).
<Br>Sample (01) : Type d'échantillon (01 : tumeur solide primaire, 11 : tissu normal).
<Br>Vial (A) : La fiole spécifique (portion) du prélèvement.
<Br>Portion (11) : La portion physique utilisée pour l'extraction.
<Br>Analyte (R) : Le type de molécule extraite (R pour l'ARN, D pour l'ADN).
<Br>Plate (A13S) : Identifie la plaque de 96 puits utilisée pour le séquençage.
<Br>Center (07) : Le centre de séquençage qui a généré la donnée.

2. Le Barcode Court (Patient / Participant)
Ce code est utilisé comme identifiant dans la Matrice Clinique et sert de référence d'identité.
<Br>Projet (TCGA) : The Cancer Genome Atlas.
<Br>TSS (EY) : Tissue Source Site (Centre de prélèvement).
<Br>Participant (A1GK) : Identifiant unique de la patiente.

Usage Stratégique : La "Common Key"
Le code à 12 caractères (TCGA-EY-A1GK) est la Clé de Jointure universelle. Son rôle est vital pour deux raisons :
- Alignement des Espaces : Il permet d'aligner chaque vecteur de l'Espace Génique (une ligne de la matrice d'expression) avec sa cible correspondante dans l'Espace Clinique (une ligne de la matrice de survie).
- Normalisation : Il permet de faire abstraction des variables techniques (fioles, plaques, centres) pour ne conserver que la signature biologique liée à l'individu.
- Règle de traitement : Pour fusionner les données, on réduit systématiquement le barcode long au format court (les 12 premiers caractères) afin de faire correspondre les gènes aux données cliniques.
</Small>

### Note Mathématique : Modélisation : De l'Espace Génique à l'Espace Clinique
<Small>
Ce document constitue le socle théorique du projet. Il détaille la transition entre la réalité biologique (comptages d'ARN) et la prédiction clinique via les structures mathématiques du Machine Learning.

1. La Matrice de Counts (Matrice d'Expression génique)
Ce tableau à deux dimensions est la porte d'entrée du modèle. Il structure les données de séquençage moléculaire :
- En lignes ($i$) : Chaque ligne représente un échantillon biologique unique (sample). Une même patiente peut être associée à plusieurs lignes si elle dispose de plusieurs prélèvements (ex : tissu tumoral et tissu sain).
- En colonnes ($j$) : Chaque colonne représente un gène spécifique (ex : TP53, BRCA1).
- Dans chaque cellule de la matrice ($x_{i,j}$) : On trouve une valeur numérique de comptage (count).

Signification biologique et Intérêt biomédical
La valeur $x_{i,j}$ indique le nombre de fragments d'ARN messager détectés pour le gène $j$ dans l'échantillon $i$. Elle reflète le niveau d'activité (l'expression génique) au moment du prélèvement.
L'ensemble des valeurs d'une même ligne constitue le profil transcriptomique de l'échantillon. Ce profil agit comme une signature biologique globale qui caractérise l'identité moléculaire de la tumeur chez une patiente donnée (état d'activation du génome et agressivité).

2. L'Équation Fondamentale de Transformation
Pour prédire l'état d'une patiente, le modèle cherche à résoudre l'équation suivante :
$$Y = F(X_1, X_2, \dots, X_n)$$

Signification des termes
$Y$ (La Variable Cible) : Une valeur numérique représentant l'Espace Clinique (ex : un score de risque ou une probabilité de survie).
$F$ (La Fonction de Transformation) : Le modèle d'IA. C'est l'opérateur mathématique qui apprend à combiner les données. En Deep Learning, on dit qu'il cherche à approximer la dépendance fonctionnelle entre les gènes et la clinique.
$(X_1, X_2, \dots, X_n)$ (Le Vecteur d'Entrée) : Les mesures contenues dans les cellules d'une ligne de la matrice. L'indice $j$ identifie le gène (la dimension de l'espace).

Pourquoi est-ce une "Intégration" ?
Le modèle ne se contente pas de corréler un gène à une maladie. Il identifie comment l'interaction entre $n$ gènes définit la signature de la pathologie.
Exemple : Le gène $X_1$ seul ou $X_2$ seul peuvent être peu informatifs. Mais si le modèle détecte que $X_1$ est haut ET $X_2$ est bas, il peut en déduire une chute drastique de la survie $Y$.

3. Structure des Données : La Hiérarchie Tensorielle
Le Machine Learning manipule des Tenseurs, une généralisation mathématique des scalaires et des matrices.

Distinction entre Ordre et Dimension
L'Ordre (ou Rang) : C'est le nombre d'indices nécessaires pour identifier une composante.
Ordre 0 : Scalaire (ex : l'âge).
Ordre 1 : Vecteur (ex : un profil patient).
Ordre 2 : Matrice (ex : la matrice d'expression complète).

La Dimension : C'est le nombre de vecteurs de la base de l'espace. Un profil patient est un tenseur d'ordre 1 évoluant dans un espace de dimension 20 000 (nombre de gènes).

Étude de cas : Augmentation de l'Ordre vs Dimension
Action
Impact Structurel
Objet (Patient unique)
Indices nécessaires

Ajouter une valeur de feature (ex: gène $n+1$)

Augmentation de la Dimension

Vecteur (Ordre 1)

1 ($j$)

Ajouter un type de feature (ex: origine Terrestre/Martien)

Augmentation de l'Ordre

Matrice (Ordre 2)

2 ($j, k$)

Note : Si l'on ajoute un axe "Type d'espèce" ($k$) pour chaque gène ($j$), le profil d'une patiente devient une grille (Ordre 2). Le jeu de données complet devient alors un tenseur d'ordre 3 ($m \times n \times k$).

4. L'Espace Latent ($Z$) : L'Espace des Mécanismes

Le modèle projette les données dans un espace intermédiaire appelé Espace Latent.

Variables Observées ($X$) : Les gènes (20 000 dimensions). C'est une réalité biologique brute et souvent bruitée.

Variables Latentes ($Z$) : Des concepts abstraits créés par le modèle.

Analogie médicale : Si les gènes sont des symptômes isolés, l'espace latent représente les syndromes ou les mécanismes biologiques (ex : "instabilité génomique").

Rôle du Machine Learning (Représentation Non-Linéaire)

Contrairement aux méthodes statistiques classiques (comme la PCA) qui font une projection linéaire ("l'ombre" des données), les réseaux de neurones peuvent tordre et plier l'espace pour découvrir des dépendances fonctionnelles non-linéaires. On parle de Representation Learning.

5. Architectures de Transformation : L'Auto-encodeur

L'auto-encodeur est une architecture en miroir utilisée pour l'extraction de signatures :

L'Encodeur ($h$) : Compresse $X$ vers l'espace latent $Z$ de dimension réduite ($d \ll n$).


$$Z = h(X)$$

Le Goulot d'étranglement (Bottleneck) : Cette étape force le modèle à ne conserver que l'essence biologique (l'ADN mathématique) de la tumeur pour pouvoir la reconstruire.

Le Décodeur ($g$) : Tente de reconstruire le profil original $\hat{X}$ à partir de $Z$.


$$\hat{X} = g(h(X))$$

Finalité Méthodologique

Débruitage : Le passage par le goulot d'étranglement élimine les fluctuations aléatoires.

Extraction : Une fois entraîné, on utilise uniquement l'encodeur pour transformer les 20 000 gènes en un petit vecteur latent $Z$ très informatif, qui servira de base à la prédiction finale de $Y$.

6. Formalisation Finale du Flux

$$\mathcal{X} \in \mathbb{R}^{m \times n} \xrightarrow{\text{Codage}} \mathcal{Z} \in \mathbb{R}^{m \times d} \xrightarrow{\text{Prédiction}} \mathcal{Y} \in \mathbb{W}$$

L'opérateur $F$ réalise ainsi un mapping entre l'Espace des Gènes (haute dimension, ordre 2) et l'Espace Clinique (basse dimension, pronostic).

Cours par l'Exemple : Le Neurone et les Réseaux
<Small>
Ce cours résume les principes fondamentaux des réseaux de neurones en suivant une progression par l'exemple, de l'unité simple à l'approximation de fonctions complexes.

1. Définition d'un Neurone Unique
Un neurone est une fonction définie en deux étapes :
Combinaison Linéaire : On multiplie les entrées ($x$) par des coefficients ($a$) : $a_1x_1 + a_2x_2 + \dots$
Activation : On applique une fonction ($\phi$ ou $h$) sur ce résultat.
Exemple concret : Un neurone à deux entrées ($x, y$) avec des coefficients $2$ et $3$ et une fonction d'activation "Heaviside" ($H$).
Calcul : $f(x, y) = H(2x + 3y)$
Résultat : La sortie vaut $1$ si $2x + 3y \geq 0$, et $0$ sinon.
Géométrie : Ce neurone sépare le plan en deux demi-plans le long de la droite $2x + 3y = 0$.

2. Réalisation de Portes Logiques
On peut paramétrer un neurone pour qu'il se comporte comme une opération logique.
Porte OU : Poids $(1, 1)$ et biais $-1$.
$f(x, y) = H(x + y - 1)$
Rend $1$ (vrai) dès que l'une des entrées vaut $1$.
Porte ET : Poids $(1, 1)$ et biais $-2$.
$f(x, y) = H(x + y - 2)$
Rend $1$ (vrai) uniquement si $x$ et $y$ valent $1$.

3. La Limitation du Neurone Unique : Le XOR
Un seul neurone ne peut pas réaliser le OU Exclusif (XOR).
Le problème : Pour le XOR, on veut que $(0,1)$ et $(1,0)$ donnent $1$, mais que $(0,0)$ et $(1,1)$ donnent $0$.
Preuve par l'absurde : Géométriquement, il est impossible de tracer une seule droite qui sépare les points "vrais" des points "faux" dans ce cas précis. On ne peut pas séparer les croix rouges des ronds bleus avec une seule ligne.

4. Plusieurs Neurones : Vers la Complexité
En réunissant plusieurs neurones par couches, on lève les limitations précédentes.
Résolution du XOR : En utilisant un réseau de 3 neurones (2 couches), on combine les résultats de deux droites pour isoler les zones du XOR.
Zones Polygonales :
Une couche de neurones définit plusieurs droites.
Le neurone de sortie (type "ET") prend l'intersection de ces demi-plans.
Cela permet de définir des zones en forme de triangle ou de polygone.
Propriétés : Avec plusieurs neurones, on peut réaliser l'union, l'intersection ou le complément d'ensembles. N'importe quelle zone délimitée par une courbe peut être approchée par un polygone, et donc par un réseau de neurones.

5. Théorème d'Approximation Universelle
Ce théorème stipule que pour toute fonction continue sur un intervalle $[a, b]$, il existe un réseau de neurones capable de l'approcher d'aussi près que l'on veut.
Le mécanisme par l'exemple (Les Créneaux) :
On crée une "marche" (Heaviside).
En combinant deux marches décalées, on crée un créneau (une fonction qui vaut $1$ sur un petit segment et $0$ ailleurs).
En additionnant une multitude de créneaux de hauteurs différentes, on peut reconstruire, "petit pas par petit pas", n'importe quelle forme de courbe.

6. Apprentissage et Pathologies
L'apprentissage consiste à trouver les bons poids par rétropropagation du gradient. Plusieurs problèmes peuvent survenir :
Modèle insuffisant : Vouloir modéliser une parabole avec une simple droite (pas assez de neurones).
Sous-apprentissage : Arrêter le calcul trop tôt (pas assez d'itérations).
Minima locaux : La descente de gradient s'arrête dans un "creux" qui n'est pas le meilleur possible. Solution : recommencer avec des poids initiaux aléatoires.

Sur-apprentissage (Overfitting) : Le modèle colle parfaitement aux points d'entraînement (apprend le bruit) mais devient incapable de faire des prédictions correctes sur de nouvelles données. La courbe devient trop complexe et "oscille" inutilement.
</Small>

### Analyse : Le Machine Learning classique est-il un neurone unique ?
<Small>
La structure mathématique $Y = F(AX + B)$ constitue la brique de base commune aux statistiques classiques et à l'intelligence artificielle moderne. Formellement, le cas du Machine Learning sans réseau multicouche revient à utiliser un neurone unique où la fonction de décision est l'opérateur lui-même.

1. La Régression Linéaire : Le neurone à activation identité
La régression linéaire est la forme la plus élémentaire de cette modélisation.
Structure : $Y = \sum(a_i x_i) + c$
Configuration : Il s'agit d'un neurone unique où la fonction d'activation $\phi$ est l'Identité ($\phi(x) = x$). L'opérateur n'applique aucune transformation non-linéaire sur la combinaison des entrées.

2. La Régression Logistique : Le neurone à activation sigmoïde
Ce modèle, standard en biostatistique pour prédire des probabilités de survie ou des diagnostics, correspond exactement à la définition d'un neurone artificiel.
Structure : $Y = \sigma(AX + B)$
Configuration : Neurone unique utilisant une fonction d'activation Sigmoïde ($\sigma(x) = \frac{1}{1 + e^{-x}}$). La sortie est contrainte dans l'intervalle $[0, 1]$.

3. Le SVM (Support Vector Machine) : Le neurone séparateur
Dans sa version linéaire, le SVM cherche l'hyperplan qui sépare deux classes avec une marge maximale.
Configuration : Il se comporte comme un neurone avec une activation de type Heaviside (fonction marche).
Différence : Contrairement au Deep Learning qui minimise une erreur moyenne par descente de gradient, le SVM optimise les poids pour maximiser la distance entre les vecteurs supports.
Limitations structurelles du neurone unique

Bien que le Machine Learning classique puisse être vu comme un neurone unique, cette structure impose deux limites majeures :

La contrainte de linéarité : Un neurone unique ne peut tracer que des frontières de décision "droites" (des hyperplans). Comme démontré par le problème du XOR, il est incapable de modéliser des relations où la séparation des classes nécessite une courbure ou une fragmentation de l'espace.

L'absence d'Espace Latent : Sans couches cachées, le modèle ne peut pas créer de variables intermédiaires. Il passe directement des variables biologiques brutes ($X$) au pronostic ($Y$), empêchant l'extraction automatique de caractéristiques abstraites (syndromes ou mécanismes tumoraux complexes).

Synthèse comparative
Modèle ML Classique
Structure
Fonction d'activation
Régression Linéaire
Neurone unique
Identité
Régression Logistique
Neurone unique
Sigmoïde
Perceptron / SVM
Neurone unique
Heaviside (Marche)
Deep Learning
Réseau de neurones
ReLU / Sigmoïde / etc.

Conclusion : Le passage du Machine Learning au Deep Learning marque la transition d'une modélisation par hyperplan (un neurone unique) à une modélisation par approximation universelle (plusieurs couches de neurones), permettant de capturer la complexité non-linéaire des données génomiques.
</Small>

## 4. Prochaines étapes
- Nettoyage des noms de gènes (Mapping ENSG -> Symbol).
- Filtrage de l'expression faible (Low counts).
- Normalisation Log-CPM.


In [33]:
import os
import pandas as pd
from datetime import datetime

path_raw = DIRS["RAW"]
f_counts = os.path.join(path_raw, "counts_samples_x_genes.csv.gz")
f_meta = os.path.join(path_raw, "metadata_clinical_merged.csv.gz")
f_anno = os.path.join(path_raw, "gene_annotation.csv.gz")

df_expr = pd.read_csv(f_counts, index_col=0, compression='gzip')
df_meta = pd.read_csv(f_meta, index_col=0, compression='gzip')
df_gene = pd.read_csv(f_anno, index_col=0, compression='gzip')

mtime_expr = datetime.fromtimestamp(os.path.getmtime(f_counts)).strftime('%Y-%m-%d %H:%M')
mtime_meta = datetime.fromtimestamp(os.path.getmtime(f_meta)).strftime('%Y-%m-%d %H:%M')
mtime_anno = datetime.fromtimestamp(os.path.getmtime(f_anno)).strftime('%Y-%m-%d %H:%M')

size_expr = os.path.getsize(f_counts)
size_meta = os.path.getsize(f_meta)
size_anno = os.path.getsize(f_anno)

print("PIPELINE 0A (ACQUISITION) STATUS REPORT : ")
print("\n✅ SUMMARY :")
print("- Acquisition des donnees brutes via TCGAbiolinks (R).")
print("- Relocalisation des fichiers dans data/raw.")

print("\n✅ DATASET LOCATION : data/raw/")
print(f"    - counts_samples_x_genes.csv.gz     | Modifie : {mtime_expr} | Taille : {size_expr} octets")
print(f"    - metadata_clinical_merged.csv.gz   | Modifie : {mtime_meta} | Taille : {size_meta} octets")
print(f"    - gene_annotation.csv.gz            | Modifie : {mtime_anno} | Taille : {size_anno} octets")

print(f"\n✅ Donnees chargees :")
print(f"    - df_expr : {df_expr.shape[0]} echantillons x {df_expr.shape[1]} genes")
print(f"    - df_meta : {df_meta.shape[0]} echantillons x {df_meta.shape[1]} colonnes")
print(f"    - df_gene : {df_gene.shape[0]} genes x {df_gene.shape[1]} colonnes")

PIPELINE 0A (ACQUISITION) STATUS REPORT : 

✅ SUMMARY :
- Acquisition des donnees brutes via TCGAbiolinks (R).
- Relocalisation des fichiers dans data/raw.

✅ DATASET LOCATION : data/raw/
    - counts_samples_x_genes.csv.gz     | Modifie : 2025-12-31 13:12 | Taille : 28775270 octets
    - metadata_clinical_merged.csv.gz   | Modifie : 2025-12-31 13:12 | Taille : 176175 octets
    - gene_annotation.csv.gz            | Modifie : 2025-12-31 13:12 | Taille : 501961 octets

✅ Donnees chargees :
    - df_expr : 553 echantillons x 60660 genes
    - df_meta : 553 echantillons x 175 colonnes
    - df_gene : 60660 genes x 2 colonnes
